## Retrieveing Data

In [2]:
import requests
import pandas as pd
import time
import json

CURRENT_TIME = round(time.time() * 1000) 

def jsonToDataFrame(json_resp):
    res = json.loads(json_resp)
    df = pd.DataFrame(res['result'])
    return df

def getSymbols():
    json_resp2 = (requests.get("https://fapi.binance.com/fapi/v1/exchangeInfo").json())
    for j in range(len(json_resp2['symbols'])):

        print(json_resp2['symbols'][j]['symbol'])

def getKlines(pair,interval,startime,endtime=CURRENT_TIME):
    query = { 
    "symbol":pair, 
    "interval": interval,
    "startTime": startime,
    "endTime": endtime,
    "limit":1000
    
    }
    # Add option for endtime
    
    json_resp = (requests.get("https://fapi.binance.com/fapi/v1/klines",params=query).json())
    df = pd.DataFrame(json_resp)
    return df


In [3]:
# Binance only gives you 1000 ticks at maximum , with this trick I extend that limit.
def dataExtender(pair,interval,initial_date,end_date): 
    list_of_dfs = []

    if interval == '15m':
        time_low = initial_date
        time_high = initial_date + (900000 *1000)

    elif interval == '1m':
        time_low = initial_date
        time_high = initial_date + (60000 *1000)

    if time_high > end_date:
        return getKlines(pair, interval, initial_date,end_date)

    while time_high < end_date:
        df = getKlines(pair, interval, time_low, time_high)
        list_of_dfs.append(df)
        time_low = time_high
        time_high += (60000 *1000)
    
    df = getKlines(pair, interval, time_low, end_date)
    list_of_dfs.append(df)
    result = pd.concat(list_of_dfs)
    
    return result
    

## Data Cleaning Aux

In [4]:
def dateToEpoch(date):
    a = date.split('/')
    timestamp = datetime(int('20' + a[2]),int(a[1]),int(a[0]),0,0).timestamp()
    return int(timestamp * 1000)

def epochToDate(epoch):
    # from datetime import datetime
    # print(datetime.fromtimestamp(int("1518308894652")/1000))
    return time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(epoch/1000))

def timestampToDay(epoch):
    day = time.strftime('%A', time.localtime(epoch))
    return day

def dayDf(date):
    # Use only with month and day. Ex: (04-20)
    temp_list = []
    for j in range(len(df) - 1):
        if date in df['open_time'].iloc[j]:
            temp_list.append(df.iloc[j])

    df = pd.DataFrame(temp_list)
    return df

def timeColumn(df):
    df_list = []
    for j in range(len(df)):
        # df['hour_of_day'].iloc[j] = int(df['open_time'].iloc[j][11:13])
        df_list.append(int(df['open_time'].iloc[j][11:13]))


    return df_list

def typeChanger(df):

    df['open_timestamp'] = df['open_timestamp'].astype('int')
    df['open'] = df['open'].astype('float')
    df['high'] = df['high'].astype('float')
    df['low'] = df['low'].astype('float')
    df['close'] = df['close'].astype('float')
    df['volume'] = df['volume'].astype('float')
    df['number_of_trades'] = df['number_of_trades'].astype('float')

    return df

def vol_amplitude_cat(df):
    vol_amplitude = df['volume'].max() - df['volume'].min()

    bins = [df['volume'].min(), vol_amplitude/4 , vol_amplitude/4*2, vol_amplitude/4*3, df['volume'].max()]
    names = ['Muy Bajo VOL', 'Medio VOL', 'Alto VOL', 'Muy Alto VOL', ]

    df['vol_cat'] = pd.cut(df['volume'], bins, labels=names)

def candleColorColumn(df):
    df['candle_color'] = [0 if df['close'].iloc[j] < df['open'].iloc[j] else 1 for j in range(len(df))]
    df['candle_color_debug'] = ["RED" if df['close'].iloc[j] < df['open'].iloc[j] else "GREEN" for j in range(len(df))]

def open_month_and_year(df):

    result_list = []
    for j in range(len(df)):
        print(df['open_time'].iloc[j][0:4])
        result_list.append(int(df['open_time'].iloc[j][0:4]))

    
    result = pd.concat(result_list,df)

    return result

def tickSize(df):
    df['tick_size'] = df['high'] - df['low']
    df['tick_size_percentage'] = df['tick_size'] * 100 / df['low']

## Data Cleaning Main

In [5]:
def dataCleaning(df):
    columns = {
        0:'open_time',
        6:'close_time',
        1:'open',
        2:'high',
        3:'low',
        4:'close',
        5:'volume',
        8:'number_of_trades'
    }

    df = df.drop(columns=[7,9,10,11])
    df = df.rename(columns = columns)
    
    # df = df.sort_values(by=['time'])
    df['open_timestamp'] = df['open_time']
    df['open_time'] = df['open_time'].apply(epochToDate)
    df['close_time'] = df['close_time'].apply(epochToDate)
    df['day_of_week'] = df['open_timestamp'].apply(timestampToDay) 
    

    # df = open_month_and_year(df)

    first_column = df.pop('close_time')
    # insert column using insert(position,column_name,
    # first_column) function
    df.insert(1, 'close_time', first_column)

    df.insert(2,'hour_of_day', timeColumn(df) )
    # this might not be an optimal approach for large datasets, but for this size it suffices

    # Transforming columns into correct type
    # timeColumn(df)
    typeChanger(df)
    candleColorColumn(df)
    # Sorting
    df = df.sort_values(by=['open_timestamp'])

    return df

## Candlestick graph

In [6]:
import plotly.graph_objects as go

import pandas as pd
from datetime import datetime

def candleGraph(df):
    fig = go.Figure(data=[go.Candlestick(x=df['time'],
                    open=df['open'],
                    high=df['high'],
                    low=df['low'],
                    close=df['close'])])

    fig.show()


## Data exploration

In [208]:
def dataPipeline(pair,interval,initial_date,end_date,verbose=False):
    if verbose == True:
        print("Retrieving data...")
    df = dataExtender(pair,interval,initial_date, end_date)
    if verbose == True:
        print('Data retrieved succesfully...')
        print('Starting cleaning...')
    df = dataCleaning(df)
    df.reset_index(drop=True ,inplace=True)
    if verbose == True:
        print('Ended cleaning...')
    return df

## Month to month

In [ ]:
lista_dfs_15m =[]
for j in range(1,12):
    print('15m')
    initial_date = dateToEpoch("28/{}/21".format(j))
    end_date = dateToEpoch("28/{}/21".format(j+1))
    print(epochToDate(initial_date),epochToDate(end_date))
    lista_dfs_15m.append(("Mes {}".format(j),dataPipeline('LUNAUSDT','15m',initial_date,end_date)))

In [ ]:
lista_dfs_1m =[]
for j in range(1,12):
    print('1m')
    initial_date = dateToEpoch("30/{}/21".format(j))
    end_date = dateToEpoch("30/{}/21".format(j+1))
    print(epochToDate(initial_date),epochToDate(end_date))
    lista_dfs_1m.append(dataPipeline('LUNAUSDT','1m',initial_date,end_date))

1m
2021-11-01 00:00:00 2021-12-30 00:00:00
Retrieving data...
Data retrieved succesfully...
Starting cleaning...
Ended cleaning...


## Whole year

In [ ]:
initial_date = dateToEpoch("1/1/21")
end_date = dateToEpoch("30/12/22")

result_15m = dataPipeline("LUNAUSDT",'15m',initial_date,end_date)
result_1m = dataPipeline("LUNAUSDT",'1m',initial_date,end_date)

Retrieving data...
Data retrieved succesfully...
Starting cleaning...
Ended cleaning...
Retrieving data...
Data retrieved succesfully...
Starting cleaning...
Ended cleaning...


## Backtesting

In [12]:
def rrrgTick(df):
    df['rrrg'] = [
        1 if 
        df['candle_color'].iloc[j-3] == 0 and 
        df['candle_color'].iloc[j-2] == 0 and 
        df['candle_color'].iloc[j-1] == 0 and 
        df['candle_color'].iloc[j] == 1
        else 0 if
        df['candle_color'].iloc[j-3] == 0 and 
        df['candle_color'].iloc[j-2] == 0 and 
        df['candle_color'].iloc[j-1] == 0 and 
        df['candle_color'].iloc[j] == 0
        else 12
        for j in range(len(df))
        ]

def grrrgTick(df):
    df['grrrg'] = [
        1 if 
        df['candle_color'].iloc[j-4] == 1 and 
        df['candle_color'].iloc[j-3] == 0 and 
        df['candle_color'].iloc[j-2] == 0 and 
        df['candle_color'].iloc[j-1] == 0 and 
        df['candle_color'].iloc[j] == 1
        else 0 if
        df['candle_color'].iloc[j-4] == 1 and 
        df['candle_color'].iloc[j-3] == 0 and 
        df['candle_color'].iloc[j-2] == 0 and 
        df['candle_color'].iloc[j-1] == 0 and 
        df['candle_color'].iloc[j] == 0
        else 12
        for j in range(len(df))
        ]

def rrrTick(df):
    df['rrr'] = [
        1 if 
        df['candle_color'].iloc[j-2] == 0 and 
        df['candle_color'].iloc[j-1] == 0 and 
        df['candle_color'].iloc[j] == 0
        else 0
        for j in range(len(df))
        ]

In [37]:
def strat1Earnings(df,pattern,initial_balance,verbose=False):
    
    rrrTick(df)

    winable_amount = 1
    winable_percent = 0
    balance = initial_balance
    fee = 0.03
    winners = 0
    loosers = 0
    operation_list = []

    for j in range(len(df)):
        
        if df[pattern].iloc[j-1] == 1 and df[pattern].iloc[j-2] != 1:
            #print(df['open_time'].iloc[j-1])
            # print(winable_percent)
            # print((df['close'].iloc[j] - df['open'].iloc[j]) * 100 / (df['open'].iloc[j])
            round_percent_delta = (df['close'].iloc[j] - df['open'].iloc[j]) * 100 / (df['open'].iloc[j])
            balance += (balance * round_percent_delta / 100) - fee
            
            if round_percent_delta > 0:
                winners += 1
                operation_list.append(('WINNER --- Compra: {} en fecha: {}'.format(df['close'].iloc[j-2],df['open_time'].iloc[j-2]), 'Venta: {} en fecha: {}'.format(df['close'].iloc[j],df['open_time'].iloc[j])))
                # print(df['open_time'].iloc[j-2] , 'WINNER')

            else:
                # print(df['open_time'].iloc[j-2] , 'LOOSER')
                loosers += 1
                operation_list.append(('LOOSER --- Compra: {}'.format(df['close'].iloc[j-2]), 'Venta: {}'.format(df['close'].iloc[j])))
                
    percentage = balance * 100 / initial_balance - 100
        
    winrate = winners / (winners + loosers) * 100
    
    result = [balance - initial_balance, percentage]
    
    if verbose == True:
        print("Pattern: " + pattern)
        print("DF has: {} elements".format(len(df)))
        print("Winners: {} --- Loosers: {} --- Winrate: {}".format(winners,loosers, winrate))
        print("Initial Balance: {} --- End Balance: {} -- Percentage: {} ".format(initial_balance, balance, percentage))     

    return result

def strat2Earnings(df,pattern,initial_balance,verbose=False):
    
    rrrTick(df)
    
    winable_amount = 1
    winable_percent = 0
    balance = initial_balance
    winners = 0
    loosers = 0
    fee = 0.03

    operation_list = []

    for j in range(len(df)):
        
        if df[pattern].iloc[j-2] == 1 and df[pattern].iloc[j-3] != 1:
            # print(winable_percent)
            # print((df['close'].iloc[j] - df['open'].iloc[j]) * 100 / (df['open'].iloc[j])
            round_percent_delta = (df['close'].iloc[j] - df['close'].iloc[j-2]) * 100 / (df['open'].iloc[j])
            balance += (balance * round_percent_delta / 100 ) - fee

            if round_percent_delta > 0:
                winners += 1
                operation_list.append(('WINNER --- Compra: {} en fecha: {}'.format(df['close'].iloc[j-2],df['open_time'].iloc[j-2]), 'Venta: {} en fecha: {}'.format(df['close'].iloc[j],df['open_time'].iloc[j])))
                # print(df['open_time'].iloc[j-2] , 'WINNER')

            else:
                # print(df['open_time'].iloc[j-2] , 'LOOSER')
                loosers += 1
                operation_list.append(('LOOSER --- Compra: {}'.format(df['close'].iloc[j-2]), 'Venta: {}'.format(df['close'].iloc[j])))

    percentage = round(balance * 100 / initial_balance - 100)
    
    winrate = winners / (winners + loosers) * 100
    
    result = [balance - initial_balance, percentage]

    if verbose == True:  
        print("Pattern: " + pattern)
        print("DF has: {} elements".format(len(df)))  
        print("Winners: {} --- Loosers: {} --- Winrate: {}".format(winners,loosers, winrate))
        print("Initial Balance: {} --- End Balance: {} -- Percentage: {} ".format(initial_balance, balance, percentage))     

    return result, operation_list

In [38]:
def stratsSimulator(initial_date,end_date):
    # initial_date = dateToEpoch("1/1/22")
    # end_date = dateToEpoch("1/3/22")

    initial_date = dateToEpoch(initial_date)
    end_date = dateToEpoch(end_date)

    result_15m = dataPipeline("LUNAUSDT",'15m',initial_date,end_date)
    result_1m = dataPipeline("LUNAUSDT",'1m',initial_date,end_date)

    data_list_1, op_list_1 = strat1Earnings(result_15m,'rrr',1000)
    data_list_2, op_list_2 = strat1Earnings(result_1m,'rrr',1000)
    data_list_3, op_list_3 = strat2Earnings(result_15m,'rrr',1000)
    data_list_4, op_list_4 = strat2Earnings(result_1m,'rrr',1000)

    print("Strat 1: 15m ", data_list_1)
    print("Strat 1: 1m ", data_list_2)
    print("Strat 2: 15m ", data_list_3)
    print("Strat 2: 1m ", data_list_4)


In [43]:
stratsSimulator("1/3/22","9/3/22")

Retrieving data...
Data retrieved succesfully...
Starting cleaning...
Ended cleaning...
Retrieving data...
Data retrieved succesfully...
Starting cleaning...
Ended cleaning...
Strat 1: 15m  [29.29647711561074, 2.929647711561074]
Strat 1: 1m  [-101.15230770937751, -10.115230770937742]
Strat 2: 15m  [18.594542559074284, 2]
Strat 2: 1m  [-60.31735836732355, -6]


In [41]:
stratsSimulator("1/2/22","1/3/22")

Retrieving data...
Data retrieved succesfully...
Starting cleaning...
Ended cleaning...
Retrieving data...
Data retrieved succesfully...
Starting cleaning...
Ended cleaning...
Strat 1: 15m  [160.05429161882603, 16.005429161882617]
Strat 1: 1m  [21.786078773726103, 2.178607877372599]
Strat 2: 15m  [272.66021436104324, 27]
Strat 2: 1m  [438.30899993791013, 44]


In [62]:
stratsSimulator("2/2/22","4/3/22")

Retrieving data...
Data retrieved succesfully...
Starting cleaning...
Ended cleaning...
Retrieving data...
Data retrieved succesfully...
Starting cleaning...
Ended cleaning...
Strat 1: 15m  [159.16422617634748, 15.916422617634737]
Strat 1: 1m  [-38.076205325311435, -3.8076205325311463]
Strat 2: 15m  [248.23450127528963, 25]
Strat 2: 1m  [345.6827888720204, 35]


In [48]:
stratsSimulator('1/12/21','1/3/22')

Retrieving data...
Data retrieved succesfully...
Starting cleaning...
Ended cleaning...
Retrieving data...
Data retrieved succesfully...
Starting cleaning...
Ended cleaning...
Strat 1: 15m  [601.2703466153964, 60.12703466153965]
Strat 1: 1m  [-335.10947405639695, -33.5109474056397]
Strat 2: 15m  [292.81025380902247, 29]
Strat 2: 1m  [1057.3209529770893, 106]


In [63]:
stratsSimulator('1/10/21','1/12/21')

Retrieving data...
Data retrieved succesfully...
Starting cleaning...
Ended cleaning...
Retrieving data...
Data retrieved succesfully...
Starting cleaning...
Ended cleaning...
Strat 1: 15m  [347.2208427825917, 34.72208427825916]
Strat 1: 1m  [-207.52640772067036, -20.75264077206704]
Strat 2: 15m  [460.8344167398441, 46]
Strat 2: 1m  [-524.9796995662409, -52]


In [151]:
from binance.spot import Spot 

client = Spot()

# Get server timestamp
print(client.time())
# Get klines of BTCUSDT at 1m interval
# print(client.klines("BTCUSDT", "1m"))
# Get last 10 klines of BNBUSDT at 1h interval
klines = client.klines("LUNAUSDT", "15m", limit=10)

# api key/secret are required for user data endpoints
client = Spot(key='TnMf0pzVavvM3R3FfYMEap5gi5U7dcDrsAyS21jCteziIZpmUVwyte5ComihUtcL', secret='aInyPpmFaj9ZO3SNwAo7aJrlIHSpAeoKparPnKUq079lnPizEhdE9P1myHI12BsN')

# Get account and balance information
# print(client.account())

# response = client.new_order(**params)

{'serverTime': 1646838522841}


In [235]:
horario = [":{}:".format(j) for j in range(10,60)]
for j in range(0,10):
    horario.append(':{}:'.format(j))

params = {
    'symbol': 'LUNAUSDT',
    'side': 'BUY',
    'type': 'LIMIT',
    'timeInForce': 'GTC',
    'quantity': 1,
    'price': 60
}

params_TP = {
    'symbol': 'LUNAUSDT',
    'side': 'BUY',
    'type': 'TAKE_PROFIT',
    'stopPrice': 65,
    'timeInForce': 'GTC',
    'quantity': 1,
    'price': 65
}

params_TP = {
    'symbol': 'LUNAUSDT',
    'side': 'BUY',
    'type': 'TAKE_PROFIT',
    'stopPrice': 65,
    'timeInForce': 'GTC',
    'quantity': 1,
    'price': 65
}

while True:
    CURRENT_TIME = round(time.time() * 1000) 
    initial_date = CURRENT_TIME - 60000 * 4
    open_order = client.get_open_orders()
                
    last_klines = dataPipeline('LUNAUSDT','1m',initial_date, CURRENT_TIME)
    rrrTick(last_klines)
    print('hola')
    print(last_klines.iloc[-1]['close'])
#and len(open_order) < 1
    if  last_klines.iloc[-1]['rrr'] == 1:
        print('NUEVA ORDEN',epochToDate(CURRENT_TIME))
        print(last_klines)
        response = client.new_order(**params)
        time.sleep(60)
    elif len(open_order) > 1:
        print('ya hay orden')
    else:
        print('no hay patron')    

    time.sleep(5)

hola
99.617
no hay patron
hola
99.497
NUEVA ORDEN 2022-03-09 17:12:33
             open_time           close_time  hour_of_day     open     high  \
0  2022-03-09 17:09:00  2022-03-09 17:09:59           17  100.063  100.100   
1  2022-03-09 17:10:00  2022-03-09 17:10:59           17   99.994  100.094   
2  2022-03-09 17:11:00  2022-03-09 17:11:59           17   99.681   99.681   
3  2022-03-09 17:12:00  2022-03-09 17:12:59           17   99.499   99.673   

      low   close   volume  number_of_trades  open_timestamp day_of_week  \
0  99.777  99.991  47854.0            3944.0   1646856540000    Thursday   
1  99.615  99.679  44168.0            3348.0   1646856600000      Friday   
2  99.322  99.500  47731.0            3916.0   1646856660000      Friday   
3  99.479  99.497  21108.0            1660.0   1646856720000    Saturday   

   candle_color candle_color_debug  rrr  
0             0                RED    1  
1             0                RED    1  
2             0                R

KeyboardInterrupt: 

In [276]:
CURRENT_TIME = round(time.time() * 1000) 
initial_date = CURRENT_TIME - 60000 * 4

k = dataPipeline('LUNAUSDT','1m',initial_date, CURRENT_TIME)

params = {
    'symbol': 'LUNAUSDT',
    'side': 'BUY',
    'type': 'LIMIT',
    'timeInForce': 'GTC',
    'quantity': 3,
    'price': 60
}

params_TP = {
  'type': 'TAKE_PROFIT_LIMIT',
  'symbol': 'LUNAUSDT',
  'side': 'SELL',
  'quantity': 3,
  'price': 120,
  'stopPrice': 90,
  'timeInForce': 'GTC',
  

}

params_SL = {
  'type': 'STOP_LOSS_LIMIT',
  'symbol': 'LUNAUSDT',
  'side': 'SELL',
  'quantity': 3,
  'price': 150,
  'stopPrice': 100,
  'timeInForce': 'GTC',
  

}

# response = client.new_order(**params)
# print(response)
response = client.new_order(**params_TP)
print(response)

response = client.new_order(**params_SL)
print(response)

{'symbol': 'LUNAUSDT', 'orderId': 1341562512, 'orderListId': -1, 'clientOrderId': '6UkRXRVAio8qpHA3tw5xX0', 'transactTime': 1646859378515}
{'symbol': 'LUNAUSDT', 'orderId': 1341562560, 'orderListId': -1, 'clientOrderId': 'Dhin6myWg5BRpHepG4ZnG7', 'transactTime': 1646859379461}
